## 添加依赖

In [19]:
import os
import json
import numpy as np
import faiss
from typing import List, Dict, Tuple
from dotenv import load_dotenv
from ollama import Client
from collections import deque
import requests

### 构建HNSW近似最近邻索引 (IndexHNSWFlat)

In [ ]:
def create_hnsw_index(dim: int, M: int = 32, efConstruction: int = 128, efSearch: int = 64) -> faiss.Index:
    """
    创建并配置HNSW索引
    
    参数:
        dim (int): 向量维度
        M (int): HNSW节点连接数（影响图结构），默认32
        efConstruction (int): 构建时的搜索深度，默认128
        efSearch (int): 查询时的搜索深度，默认64
        
    返回:
        faiss.Index: 配置好的HNSW索引
    """
    # 创建HNSW Flat索引（使用L2距离）
    index = faiss.IndexHNSWFlat(dim, M)
    
    # 设置构建参数
    index.hnsw.efConstruction = efConstruction
    
    # 设置查询参数
    index.hnsw.efSearch = efSearch
    
    # 启用构建日志
    index.verbose = True
    
    print(f"HNSW索引创建完成: M={M}, efConstruction={efConstruction}, efSearch={efSearch}")
    
    return index

### 加载知识数据，包含 .index, _chunks.json, _vectors.json 三个文件
### 并且将文本与向量对应起来，返回FAISS索引和文本元数据列表。

In [21]:
def load_data_group(folder_path: str) -> Tuple[faiss.Index, List[Dict]]:
    files = os.listdir(folder_path)
    # 找出所有.index文件，依据命名前缀找到对应.json文件
    index_files = [f for f in files if f.endswith('.index')]

    # 结果：合并所有组的FAISS索引和对应元数据
    # 为了合并多个FAISS索引，我们这里采取一次性加载所有embedding并重建索引的方案（便于统一检索）
    all_embeddings = []
    all_metadata = []

    for idx_file in index_files:
        prefix = idx_file[:-6]  # 去掉 '.index'
        chunks_file = prefix + "_chunks.json"
        vectors_file = prefix + "_vectors.json"

        idx_path = os.path.join(folder_path, idx_file)
        chunks_path = os.path.join(folder_path, chunks_file)
        vectors_path = os.path.join(folder_path, vectors_file)

        # 检查对应文件是否存在
        if not (os.path.exists(chunks_path) and os.path.exists(vectors_path)):
            print(f"警告：文件组不完整，缺少 {chunks_file} 或 {vectors_file}，跳过 {prefix}")
            continue

        # 读取文本块
        with open(chunks_path, "r", encoding="utf-8") as f:
            chunks_data = json.load(f)

        # 读取向量数据
        with open(vectors_path, "r", encoding="utf-8") as f:
            vectors_data = json.load(f)

        # 将文本和向量对应起来，确保id对应一致
        id_to_text = {item['id']: item['text'] for item in chunks_data}

        for vec_item in vectors_data:
            vec_id = vec_item['id']
            embedding = vec_item.get('embedding')
            if embedding is None:
                print(f"警告：id={vec_id}在 {vectors_file} 中缺少embedding，跳过")
                continue
            text = id_to_text.get(vec_id, "")
            all_embeddings.append(embedding)
            all_metadata.append({
                "id": vec_id,
                "text": text,
                "source_file": vec_item.get("source_file", ""),
                "prefix": prefix
            })

    # 构建FAISS索引 - 使用 IndexHNSWFlat
    if len(all_embeddings) == 0:
        raise ValueError("没有加载到任何embedding向量，无法构建索引")

    dimension = len(all_embeddings[0])
    
    # 创建HNSW索引
    index = create_hnsw_index(dimension)
    
    # 添加向量到索引
    embeddings_np = np.array(all_embeddings, dtype=np.float32)
    index.add(embeddings_np)
    
    return index, all_metadata

### embedding向量化文本

In [ ]:
client = Client(host='http://localhost:11434')
 
def get_embedding(text, model="dengcao/Qwen3-Embedding-8B:Q5_K_M"):
    response = client.embeddings(model=model, prompt=text)
    return np.array(response['embedding'], dtype=np.float32)

### RAG 引擎类，检索生成问答

In [23]:
class RAGSystem:
    def __init__(self, index_path, temperature=0.8, model_name="qwen3:8b"):
        self.index, self.metadata = load_data_group(index_path)
        self.chat_history = deque(maxlen=6)  # 保留3轮对话
        self.temperature = temperature  # 存储温度值
        self.model_name = model_name  # 生成模型名称
    
    def _retrieve(self, query: str, top_k=5):
        """向量检索"""
        emb = get_embedding(query)
        distances, indices = self.index.search(np.array([emb]), top_k)
        return [self.metadata[idx] for idx in indices[0]]
    
    def _build_prompt(self, context_docs, query):
        """知识增强的Prompt构造"""
        context = "\n\n".join(f"【知识片段 {i}】{doc['text']}" 
                             for i, doc in enumerate(context_docs))
        return f"背景知识：\n{context}\n\n问题：{query}\n回答要求：用中文回答，包含知识来源"
    
    def generate_answer(self, query, top_k):
        # 1. 检索上下文
        context_docs = self._retrieve(query,top_k)


        # 2. 构造Prompt（包含多轮历史）
        prompt = "你是有专业文档支持的助手。请根据以下知识和对话历史回答问题：\n\n"
    
        # 添加对话历史（如果有）
        if self.chat_history:
            for i, content in enumerate(self.chat_history):
                role = "用户" if i % 2 == 0 else "助手"
                prompt += f"{role}: {content}\n"
    
        # 添加当前问题和知识片段
        prompt += f"\n知识背景：\n{self._build_prompt(context_docs, query)}"
        print("=== 最终 prompt ===")
        print(prompt)
        # 3. 调用Ollama
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={
                "model": self.model_name,
                "prompt": prompt,
                "stream": False,  # 明确要求非流式输出
                "options": {
                    "temperature": self.temperature  # 设置温度值
                }
            }
        )
        answer = response.json()["response"]
        
        # 4. 后处理
        clean_answer = self._postprocess(answer)
        self.chat_history.extend([query, clean_answer])
        return clean_answer


    def _postprocess(self, text):
        """答案优化"""
        last_query = self.chat_history[-2] if len(self.chat_history) >= 2 else ""
        # 学术文献添加引用
        if any(kw in last_query  for kw in ["文献", "研究"]):
            return f"{text}\n\n来源：{self.current_sources()}"
            
        # 法律文件精确条款标注
        elif "条款" in last_query :
            return re.sub(r"第([零一二三四五六七八九十百]+)条", r"【\g<0>】", text)
        
        return text.strip()

### 主程序

In [24]:
if __name__ == "__main__":
    folder = "/workspace/code/datas/output"  # 修改为你的文件夹路径
    rag_engine = RAGSystem(folder)

    
    q = '总结一下民国文化的特点'
    print("欢迎使用AI知识脑\n")
    ans = rag_engine.generate_answer(q, top_k=5)
    print("\n=== 回答 ===\n")
    from IPython.display import display, HTML
    display(HTML("<style>pre { white-space: pre-wrap; }</style>"))
    print(ans)

HNSW索引创建完成: M=32, efConstruction=128, efSearch=64
hnsw_add_vertices: adding 2891 elements on top of 0 (preset_levels=0)
  max_level = 2
Adding 2 elements at level 2
Adding 73 elements at level 1
Adding 2816 elements at level 0
Done in 640.215 ms
欢迎使用AI知识脑

=== 最终 prompt ===
你是有专业文档支持的助手。请根据以下知识和对话历史回答问题：


知识背景：
背景知识：
【知识片段 0】化，它在专制主义压抑下崛起，在同帝国主义和封建买办文化的搏争中壮大，直至确立在全国范围内的主导地位。 其长现之三在于：民国社会复杂的阶级构成，特别是政党斗争和政权分割的多元政治形势，决定了民国文化的多元格局，即多种文化并存、两大文化阵营对垒的局面；同时也赋了了它鲜明的政治性格。 在整个民国时期，社会结构呈现出十分复杂的状态，作为各个社会阶级和阶层意识形态表现的文化也同样呈现出比晚清时期更为错综复杂的构成。毛泽东在《新民主主义论》小，曾将此期的文化成分概括为三种：（1）“帝国主义文化”，－-切包含双化思想的文化，都属于这一类：（2）“半封建文化"，凡属上张尊孔读经、提倡旧礼教、旧思想、反对新文化新思想者，都属于这-类;（3）新民主主义的文化，即“无产阶级领导的反帝反封建的文化”。这当然是就其主要成分而言，如果分得细些，还有更多。

【知识片段 1】国最杰出的抒情诗人”。② 在诗歌创作中取得更大成就的是新月社成员，其代表人物是徐志摩、闻一多和朱湘。徐志摩（1891一1931年）是“新月诗派”的盟主，在1922年到1931年近十年的时间里，先后出版了《志摩的诗》、《翡冷翠的一夜》、《猛虎集》等诗集，《落叶》、《自剖》、《巴黎的鳞爪》等散文集。收在《志摩的诗》中的早期诗作大都内容健康，格调清新，形式活泼白然，如《落叶小唱》、《残诗》等，洋溢着积极乐观的情调。但面对当时中国社会的黑 暗，诗人也在许多诗篇中流露出失望和颓唐情绪，特别是在《翡冷翠的·夜》和《猛虎集》中的一些诗篇，如《大帅》、《三月「二深夜大沽口外》等。此后，他以全部精力

<think>
好的，我现在需要帮用户总结民国文化的特点。首先，我得仔细看看用户提供的知识片段，确保准确理解每个部分的信息。用户提到要包含知识来源，所以我得注意每个要点对应的片段编号。

首先，知识片段0提到民国文化的多元格局，分为帝国主义文化、半封建文化和新民主主义文化，还有社会结构复杂导致多种文化并存。这应该作为多元文化并存的点，引用片段0。

接下来，知识片段1讲的是新月社的诗人，比如徐志摩，他们的诗歌风格和主题变化，但这里可能更多是文学方面的例子，不过用户的问题是关于整体文化特点，可能需要确认是否相关。不过片段1可能更多是文学例子，可能暂时不直接作为文化特点，但可以提到文学的多样性。

知识片段2讨论的是秘密宗教和伦理道德的变化，提到新旧伦理道德的冲突，这可能属于伦理道德变革的部分，引用片段2。

知识片段3讲的是宗教的中国化，比如天主教和新教的本色化运动，这属于宗教文化的中国化，引用片段3。

知识片段4涉及新儒家哲学的发展，结合中西哲学，这应该作为哲学思想的融合，引用片段4。

现在需要将这些点组织起来。用户的问题是关于民国文化的特点，所以需要涵盖政治、社会、宗教、文学、哲学等方面。根据知识片段0，政治方面有多元文化格局，两大阵营对垒；社会结构复杂；宗教方面有中国化倾向；哲学上有新儒家的发展；伦理道德的变革，新旧冲突。

需要注意的是，用户可能希望每个特点都有对应的来源引用，所以每个要点都要标注对应的片段编号。比如多元文化格局来自片段0，宗教中国化来自片段3，哲学思想融合来自片段4，伦理道德变革来自片段2。

另外，是否要提到文学成就？片段1提到徐志摩等人的诗歌，但可能属于文化的一个方面，但用户的问题可能更偏向整体文化特点，所以可能不需要单独列出文学，但可以作为文化多样性的一个例子。

最后，整合这些要点，确保每个特点都有对应的来源，并且语言简洁明了。
</think>

民国文化的特点可总结如下：  
1. **多元文化并存与政治性鲜明**：民国时期社会结构复杂，形成多种文化并存的局面，主要分为帝国主义文化、半封建文化和新民主主义文化三大阵营。这种多元格局赋予文化鲜明的政治性格，如毛泽东在《新民主主义论》中将其概括为三类（知识片段0）。  
2. **宗教文化中国化**：天主教和新教等宗教出现明显中国化倾向，如采用中国传统节日、民族曲调唱赞美诗、融合佛